In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

!pip install autogluon.tabular[all] 

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/machine-failure-predictions/machine failure.csv
/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 104.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0

In [2]:
############################### FEATURE ENGINER V2 #######################################################


# Read in the data
filepath = '/kaggle/input/playground-series-s3e17'

df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col=[0])
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col=[0])
original = pd.read_csv('/kaggle/input/machine-failure-predictions/machine failure.csv', index_col=[0])
target_col = 'Machine failure'
# Stack training and original data
train = pd.concat([df_train, original])
test = df_test.copy()

X_train = train.drop([f'{target_col}'],axis=1).reset_index(drop=True)
y_train = train[f'{target_col}'].reset_index(drop=True)
X_test = test.reset_index(drop=True)


def create_features(df):
    df["Temperature ratio"] = df['Process temperature [K]'] / df['Air temperature [K]']
    df['Power']= df['Torque [Nm]']*df['Rotational speed [rpm]']
    df['Torque * Tool wear'] = df['Torque [Nm]'] * df['Tool wear [min]']
    df['TotalFailures'] = df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1)
    new_cols = [
        #'Temperature difference [K]', 
        'Power', 
        'Temperature ratio',
        'Torque * Tool wear','TotalFailures']
    return df, new_cols

def rename(df):
    df.columns = df.columns.str.replace('\[.*?\]', '', regex=True)
    df.columns = df.columns.str.strip()
    # df.columns = df.columns.str.replace(' ', '')
    
    return df

X_train, _ = create_features(X_train)
X_test, new_cols = create_features(X_test)
# Rename
X_train = rename(X_train)
X_test = rename(X_test)

In [3]:
from autogluon.tabular import TabularPredictor, TabularDataset
train = X_train
train["label"] = y_train
train = TabularDataset(train)

In [4]:
excluded_model_types = ['KNN', 'NN_TORCH']
predictor2 = TabularPredictor(label="label", problem_type='binary',verbosity=-1,
                             eval_metric='log_loss',sample_weight='balance_weight').fit(train,presets="best_quality",excluded_model_types=excluded_model_types,
                                                                    auto_stack=True,save_space=True,
                                                                    num_stack_levels=2, num_bag_folds=3,num_bag_sets=6,time_limit=1800)
# ['accuracy', 'acc', 'balanced_accuracy', 'mcc', 'roc_auc_ovo_macro', 'log_loss', 'nll', 'pac', 'pac_score', 'quadratic_kappa', 'roc_auc', 'average_precision', 'precision', 'precision_macro', 'precision_micro',
#  'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_weighted']

In [5]:
predictor2.leaderboard()
#  -0.016092 

                      model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L4  -0.014840     104.853632  1597.828476                0.006743           7.167696            4       True         33
1       WeightedEnsemble_L3  -0.016403      71.541521  1302.921845                0.006763           6.822583            3       True         22
2    NeuralNetFastAI_BAG_L3  -0.016672      81.266489  1375.751399                4.503833          34.728991            3       True         30
3       WeightedEnsemble_L2  -0.018602      32.334250   686.043019                0.007020           7.141581            2       True         11
4    NeuralNetFastAI_BAG_L2  -0.019080      43.850881   897.811802                4.043177         179.576239            2       True         19
5   RandomForestEntr_BAG_L3  -0.020286      82.209345  1398.145907                5.446689          57.123500            3       T

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L4,-0.014840,104.853632,1597.828476,0.006743,7.167696,4,True,33
1,WeightedEnsemble_L3,-0.016403,71.541521,1302.921845,0.006763,6.822583,3,True,22
2,NeuralNetFastAI_BAG_L3,-0.016672,81.266489,1375.751399,4.503833,34.728991,3,True,30
3,WeightedEnsemble_L2,-0.018602,32.334250,686.043019,0.007020,7.141581,2,True,11
4,NeuralNetFastAI_BAG_L2,-0.019080,43.850881,897.811802,4.043177,179.576239,2,True,19
5,RandomForestEntr_BAG_L3,-0.020286,82.209345,1398.145907,5.446689,57.123500,3,True,26
6,ExtraTreesGini_BAG_L3,-0.020547,82.967353,1363.240935,6.204696,22.218527,3,True,28
7,ExtraTreesEntr_BAG_L3,-0.021191,82.837199,1363.719904,6.074543,22.697497,3,True,29
8,RandomForestGini_BAG_L3,-0.021501,82.233539,1402.045410,5.470883,61.023003,3,True,25
9,NeuralNetFastAI_BAG_L1,-0.021882,4.599208,266.244445,4.599208,266.244445,1,True,8


In [6]:
#y_pred_auc = predictor.predict_proba(X_test, as_pandas=False, as_multiclass=False)
y_pred_logloss = predictor2.predict_proba(X_test, as_pandas=False, as_multiclass=False)

In [7]:
y_pred_logloss

array([0.00018877, 0.0001821 , 0.00014927, ..., 0.0001874 , 0.00793444,
       0.00014059])

In [8]:

#models = ['WeightedEnsemble_L3','WeightedEnsemble_L2']

# Inicialize uma matriz vazia para armazenar as previsões de cada modelo
#predictions = []

# Faça previsões para cada modelo
#for models in models:
    # Realize a previsão para o modelo atual
#    model_pred = predictor.predict_proba(X_test, model=models,as_pandas=False, as_multiclass=False)
    # Adicione as previsões do modelo atual à matriz
#    predictions.append(model_pred)

# Calcule a média das previsões de todos os modelos
#average_prediction = np.mean(predictions, axis=0)

# Exiba a média das previsões
#print("Average Prediction:", average_prediction)
#Average Prediction: [0.03733805 0.06888288 0.0210157  ... 0.04945559 0.02032589 0.03096515]

In [9]:


#ypredmedia = (y_pred_auc+y_pred_logloss+average_prediction)/3
def make_submission(test_predss, prefix=''):
    sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
    sub[f'{target_col}'] = test_predss
    sub.to_csv(f'{prefix}submission.csv', index=False)
    return  sub
#sub = make_submission(np.where(ypredmedia >= 0.50, 1, 0), prefix='')
sub = make_submission(y_pred_logloss, prefix='')
sub

,id,Machine failure
0,136429,0.000189
1,136430,0.000182
2,136431,0.000149
3,136432,0.001028
4,136433,0.000145
...,...,...
90949,227378,0.000258
90950,227379,0.003502
90951,227380,0.000187
90952,227381,0.007934
